# **INTRODUCTION TO DATA SCIENCE**
## Final Projects - House Price Prediction - 22KDL

### **ABTRACT** 
This project aims to develop an accurate house price prediction model using data collected from the website https://batdongsan.vn/ban-nha/. By applying both learned and newly researched data analysis methods and techniques, we evaluated the effectiveness of various predictive models.

### **INTRODUCTION** 
In the context of an increasingly volatile real estate market, accurate house price prediction is an urgent need. The goal of this project is to build a house price prediction model based on online data, providing users with clear insights and accurate forecasts about real estate values.

### **LITERATURE REVIEW**
Previous studies have employed various methods for house price prediction, ranging from simple regression models to complex machine learning techniques such as Random Forest and Gradient Boosting. This study builds upon and further develops these proven methods.




## **DATA AND METHODS**

### **Data Collecction**

### **Data Preprocessing and Feature Engineering**

### **Kernel Density estimation (KDE)**

### **Methodology**

### **Results**

### **Discussion**

### **Conclusion and Recommendations**

### **References**

1. https://www.zenrows.com/blog/web-crawler-python#initial-crawling-script
2. https://viblo.asia/p/web-crawling-voi-thu-vien-beautifulsoup-1VgZvNGOZAw
3. https://realpython.com/beautiful-soup-web-scraper-python/
4. https://www.topcoder.com/thrive/articles/web-scraping-with-beautiful-soup
5. https://www.w3schools.com/python/python_regex.asp
6. https://docs.python.org/3/howto/regex.html
7. https://peps.python.org/pep-0008/#function-and-variable-names
8. https://cs230.stanford.edu/syllabus/
9. https://www.geeksforgeeks.org/cross-validation-machine-learning/
10. https://miai.vn/2021/01/18/k-fold-cross-validation-tuyet-chieu-train-khi-it-du-lieu/
11. https://machinelearningcoban.com/tabml_book/ch_data_processing/eda.html#
12. https://www.ibm.com/topics/exploratory-data-analysis
13. https://scikit-learn.org/stable/modules/density.html
14. https://viblo.asia/p/danh-gia-cac-mo-hinh-hoc-may-RnB5pp4D5PG
15. https://www.geeksforgeeks.org/k-nearest-neighbours/

### **About us**

##### **Members**
| **Name**| **Major**| **University**|
|-|-|-|
| Le Thanh Thuy | Data Science  | University of Science (VNUHCM) |
| Nguyen Ngoc Thanh Thu | Data Science  | University of Science (VNUHCM) |
| Phan Binh Phuong | Data Science  | University of Science (VNUHCM) |
| Luong Thanh Nam| Data Science  | University of Science (VNUHCM) |
